In [1]:
# Install dependencies (only run once)
!pip install kaggle mediapipe opencv-python scikit-learn matplotlib seaborn tqdm

  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
Using cached kaggle-1.7.4.5-py3-none-any.whl (181 kB)
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   - -------------------------------------- 1.6/51.0 MB 7.6 MB/s eta 0:00:07
   -- ----------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\navee\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [6]:
import os
import zipfile
import time

# Ensure .kaggle directory exists
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Move kaggle.json to correct location (adjust path if needed)
if not os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
    !mv kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d lmzintgraf/hand-gesture-recognition-dataset

zip_filename = 'hand-gesture-recognition-dataset.zip'

# Print files in current directory after download for debugging
print("Files in current directory after download:", os.listdir('.'))

# Wait a bit to ensure download is complete
for _ in range(10):
    if os.path.exists(zip_filename):
        break
    time.sleep(1)
else:
    # Print files for debugging
    print("Files in current directory:", os.listdir('.'))
    raise FileNotFoundError(f"{zip_filename} not found after download. Please check Kaggle authentication and ensure the file is downloaded successfully.")

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('data')

'mv' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\navee\AppData\Local\Programs\Python\Python312\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "C:\Users\navee\AppData\Local\Programs\Python\Python312\Lib\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\navee\AppData\Local\Programs\Python\Python312\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\navee\AppData\Local\Programs\Python\Python312\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_ka

Files in current directory after download: ['dogs-vs-cats', 'house-prices-advanced-regression-techniques', 'Mall_Customers.csv', 'Mall_Customers_Clustered.csv', 'sct_ml-02.ipynb', 'sct_ml_01.ipynb', 'sct_ml_03.ipynb', 'sct_ml_04.ipynb', 'submission_ridge.csv']
Files in current directory: ['dogs-vs-cats', 'house-prices-advanced-regression-techniques', 'Mall_Customers.csv', 'Mall_Customers_Clustered.csv', 'sct_ml-02.ipynb', 'sct_ml_01.ipynb', 'sct_ml_03.ipynb', 'sct_ml_04.ipynb', 'submission_ridge.csv']


FileNotFoundError: hand-gesture-recognition-dataset.zip not found after download. Please check Kaggle authentication and ensure the file is downloaded successfully.

In [9]:
# Install mediapipe if not already installed
%pip install mediapipe

# Imports
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl (413 kB)
Using cached sounddevice-0.5.2-py3-none-win_amd64.whl (363 kB)
Using cached jax-0.6.1-py3-none-any.whl (2.4 MB)
Using cached jaxlib-0.6.1-

  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\navee\\OneDrive\\Documents\\miniproject\\.conda\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Extract keypoints from hand image
mp_hands = mp.solutions.hands

def extract_keypoints(image_path):
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
        res = hands.process(img)
        if res.multi_hand_landmarks:
            lm = res.multi_hand_landmarks[0].landmark
            return np.array([[p.x, p.y, p.z] for p in lm]).flatten()
    return None

In [ ]:
# Load all images and extract features
def load_dataset(data_dir):
    X, y, labels = [], [], sorted(os.listdir(data_dir))
    for idx, label in enumerate(labels):
        folder = os.path.join(data_dir, label)
        for img in tqdm(os.listdir(folder), f"Loading {label}"):
            pts = extract_keypoints(os.path.join(folder, img))
            if pts is not None:
                X.append(pts)
                y.append(idx)
    return np.array(X), np.array(y), labels

X, y, class_names = load_dataset('data')

In [ ]:
# Train/test split and SVM training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
scaler = StandardScaler().fit(X_train)
X_train_s, X_test_s = scaler.transform(X_train), scaler.transform(X_test)

clf = SVC(kernel='rbf', C=10).fit(X_train_s, y_train)
y_pred = clf.predict(X_test_s)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
plt.show()